In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [3]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [5]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)

In [7]:
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]], dtype=int32)

In [8]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [9]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [10]:
one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [11]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [12]:
one_hot_labels[0]

array([0., 0., 0., 0., 1., 0.])

In [13]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

C:\Users\hpcnd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.3841 - loss: 1.5299 - val_accuracy: 0.7284 - val_loss: 0.8622
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - accuracy: 0.8614 - loss: 0.4434 - val_accuracy: 0.8256 - val_loss: 0.5203
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 54ms/step - accuracy: 0.9829 - loss: 0.0685 - val_accuracy: 0.8278 - val_loss: 0.5557
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - accuracy: 0.9947 - loss: 0.0263 - val_accuracy: 0.8322 - val_loss: 0.5683
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - accuracy: 0.9966 - loss: 0.0187 - val_accuracy: 0.8297 - val_loss: 0.6143
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - accuracy: 0.9978 - loss: 0.0105 - val_accuracy: 0.8284 - val_loss: 0.6395
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - accuracy: 0.9984 - loss: 0.0085 - val_accuracy: 0.8259 - val_loss: 0.6962
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 16s 39ms/step - accuracy: 0.9977 - loss: 0.0103 - 

In [21]:
input_text = "i propose a girl"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
sadness


In [22]:
model.save('emotion.h5')  # HDF5 format


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.models import load_model
import joblib

# Step 1: Load and Preprocess Data
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print("Sample Data:")
print(data.head())

texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

# Step 2: Build the Model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(set(labels)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 4: Save Model and Tokenizer
model.save('emotion_model.h5')
joblib.dump(tokenizer, 'tokenizer.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
print("Model and tokenizer saved.")


Sample Data:
                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


C:\Users\hpcnd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 29ms/step - accuracy: 0.3623 - loss: 1.5535 - val_accuracy: 0.6388 - val_loss: 1.0416
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.8034 - loss: 0.5991 - val_accuracy: 0.8156 - val_loss: 0.5559
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.9742 - loss: 0.1123 - val_accuracy: 0.8141 - val_loss: 0.5779
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.9918 - loss: 0.0388 - val_accuracy: 0.8128 - val_loss: 0.6398
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.9953 - loss: 0.0247 - val_accuracy: 0.8169 - val_loss: 0.6468
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.9971 - loss: 0.0139 - val_accuracy: 0.8144 - val_loss: 0.6656
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.9967 - loss: 0.0132 - val_accuracy: 0.8141 - val_loss: 0.7084
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.9969 - loss: 0.0128 - 

Model and tokenizer saved.


In [3]:
model = load_model('emotion_model.h5')
tokenizer = joblib.load('tokenizer.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Step 6: Define Inference Function
def predict_emotion(text, tokenizer, model, max_length, label_encoder):
    """
    Predict the emotion of a given text input.

    Args:
        text (str): Input text to classify.
        tokenizer (Tokenizer): Fitted tokenizer for preprocessing.
        model (Sequential): Trained Keras model for prediction.
        max_length (int): Maximum length used during model training.
        label_encoder (LabelEncoder): Encoder for decoding emotion labels.

    Returns:
        str: Predicted emotion label.
    """
    # Tokenize and pad the input text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    # Predict emotion
    prediction = model.predict(padded_sequence)
    emotion_index = prediction.argmax()  # Get the index of the highest probability
    emotion_label = label_encoder.inverse_transform([emotion_index])[0]  # Decode to emotion label
    return emotion_label


In [35]:
if __name__ == "__main__":
    sample_text = "hi my name is suraj , i want hridhya ,but akhil trying to steal her from me , i hate him"
    predicted_emotion = predict_emotion(sample_text, tokenizer, model, max_length, label_encoder)
    print(f"Sample Text: {sample_text}")
    print(f"Predicted Emotion: {predicted_emotion}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Sample Text: hi my name is suraj , i want hridhya ,but akhil trying to steal her from me , i hate him
Predicted Emotion: sadness


In [22]:
def show_song(emotion):
    """
    Display a YouTube link based on the emotion.

    Args:
        emotion (str): Predicted emotion label.

    Returns:
        None
    """
    emotion_songs = {
        "fear": "https://www.youtube.com/watch?v=fear_song_link",
        "joy": "https://www.youtube.com/watch?v=joy_song_link",
        "sadness": "https://www.youtube.com/watch?v=sadness_song_link",
        "anger": "https://www.youtube.com/watch?v=anger_song_link"
    }
    link = emotion_songs.get(emotion, "No song available for this emotion.")
    print(f"Recommended song for {emotion}: {link}")

# Step 8: Test Inference
if __name__ == "__main__":
    sample_text = "I am feeling so joyful today!"
    predicted_emotion = predict_emotion(sample_text, tokenizer, model, max_length, label_encoder)
    print(f"Sample Text: {sample_text}")
    print(f"Predicted Emotion: {predicted_emotion}")
    show_song(predicted_emotion)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Sample Text: I am feeling so joyful today!
Predicted Emotion: joy
Recommended song for joy: https://www.youtube.com/watch?v=joy_song_link
